<a href="https://colab.research.google.com/github/Natural-Language-Processing-YU/M3_Assignment/blob/main/scripts/m3_assignment_part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [1]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
None
### END CODE HERE ###

# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources

In [2]:
!pip install tensorflow_datasets
!pip install -U tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [3]:
pip install tensorflow

In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the names
male_names = ['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil']
female_names = ['Alice', 'Alicia', 'Alina', 'Alison', 'Alissa', 'Allyson', 'Alma', 'Althea', 'Alva', 'Alyson', 'Alyssa', 'Amber', 'Amelia', 'Amie', 'Amy', 'Ana', 'Anastasia', 'Andrea', 'Angel', 'Angela', 'Angelia', 'Angelica', 'Angelina', 'Angeline']
all_names = male_names + female_names
labels = torch.tensor([0]*len(male_names) + [1]*len(female_names))

# Tokenize names
inputs = tokenizer(all_names, padding=True, truncation=True, max_length=10, return_tensors="pt")

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_function = torch.nn.CrossEntropyLoss()

def train_model(model, inputs, labels, optimizer, loss_function):
    model.train()
    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}: Training Loss = {loss.item()}')

# Train the model
train_model(model, inputs, labels, optimizer, loss_function)

# Predict the gender of the names
model.eval()
with torch.no_grad():
    logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=1)

# Output the results
print("Name and Predicted Gender:")
for name, prediction in zip(all_names, predictions):
    print(f'{name}: {"male" if prediction == 0 else "female"}')




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Training Loss = 0.8135320544242859
Epoch 2: Training Loss = 0.7213786244392395
Epoch 3: Training Loss = 0.7056490778923035
Epoch 4: Training Loss = 0.6642819046974182
Epoch 5: Training Loss = 0.6659600138664246
Epoch 6: Training Loss = 0.6211572885513306
Epoch 7: Training Loss = 0.5677432417869568
Epoch 8: Training Loss = 0.5438061356544495
Epoch 9: Training Loss = 0.5142210125923157
Epoch 10: Training Loss = 0.3959195017814636
Epoch 11: Training Loss = 0.33589187264442444
Epoch 12: Training Loss = 0.3026914894580841
Epoch 13: Training Loss = 0.2981669306755066
Epoch 14: Training Loss = 0.2600216865539551
Epoch 15: Training Loss = 0.2689298689365387
Epoch 16: Training Loss = 0.205106720328331
Epoch 17: Training Loss = 0.1838071644306183
Epoch 18: Training Loss = 0.161748468875885
Epoch 19: Training Loss = 0.23426121473312378
Epoch 20: Training Loss = 0.1763152927160263
Name and Predicted Gender:
Abbey: male
Abbie: male
Abby: male
Addie: male
Adrian: male
Adrien: male
Ajay: mal

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(num_features, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1])
        return out

# Encoding names as one-hot vectors
def encode_names(names, max_length):
    char_to_idx = {ch: idx for idx, ch in enumerate('abcdefghijklmnopqrstuvwxyz')}
    encoded = torch.zeros((len(names), max_length, 26))
    for idx, name in enumerate(names):
        name = name.lower()
        for char_idx, char in enumerate(name):
            if char in char_to_idx:
                encoded[idx, char_idx, char_to_idx[char]] = 1
    return encoded

# List of names and their labels
male_names = ['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis', 'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel', 'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil']
female_names = ['Alice', 'Alicia', 'Alina', 'Alison', 'Alissa', 'Allyson', 'Alma', 'Althea', 'Alva', 'Alyson', 'Alyssa', 'Amber', 'Amelia', 'Amie', 'Amy', 'Ana', 'Anastasia', 'Andrea', 'Angel', 'Angela', 'Angelia', 'Angelica', 'Angelina', 'Angeline']
all_names = male_names + female_names
labels = [0] * len(male_names) + [1] * len(female_names)

# Preprocess names data
max_len = max(len(name) for name in all_names)
encoded_names = encode_names(all_names, max_len)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(encoded_names, labels, test_size=0.2, random_state=42)

# Create Tensor datasets and loaders
train_data = TensorDataset(X_train, torch.tensor(y_train, dtype=torch.long))
test_data = TensorDataset(X_test, torch.tensor(y_test, dtype=torch.long))
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1)

# Function to train the LSTM model
def train_model(model, train_loader, optimizer, criterion, num_epochs=20):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            optimizer.zero_grad()  # Clear the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update model parameters
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Function to evaluate the LSTM model
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0
    with torch.no_grad():  # Disable gradient computation
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f'Accuracy: {accuracy * 100:.2f}%')

# Initialize the model, optimizer, and loss function
input_size = 26  # As each character is one-hot encoded into a vector of length 26
hidden_size = 64
num_layers = 1
num_classes = 2

model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
train_model(model, train_loader, optimizer, criterion)

# Evaluate the model on the test set
evaluate_model(model, test_loader)



Epoch 1/20, Loss: 0.6929980874061584
Epoch 2/20, Loss: 0.6882671117782593
Epoch 3/20, Loss: 0.6868249297142028
Epoch 4/20, Loss: 0.6849748373031617
Epoch 5/20, Loss: 0.6844630599021911
Epoch 6/20, Loss: 0.682830536365509
Epoch 7/20, Loss: 0.6817387104034424
Epoch 8/20, Loss: 0.6791081070899964
Epoch 9/20, Loss: 0.6769961714744568
Epoch 10/20, Loss: 0.6755481362342834
Epoch 11/20, Loss: 0.6774375557899475
Epoch 12/20, Loss: 0.6720068454742432
Epoch 13/20, Loss: 0.6646776080131531
Epoch 14/20, Loss: 0.6491972923278808
Epoch 15/20, Loss: 0.6313737392425537
Epoch 16/20, Loss: 0.5858804941177368
Epoch 17/20, Loss: 0.5212947487831116
Epoch 18/20, Loss: 0.4426823198795319
Epoch 19/20, Loss: 0.37675228118896487
Epoch 20/20, Loss: 0.31617157384753225
Accuracy: 80.00%
